# SPARQL Tutorial für FEDLEX

## Einführung

Diese Webseite ist ein sogenanntes **interaktives JupyterLite-Notebook**. In diesem Notizbuch können Sie den Inhalt der einzelnen Zellen interaktiv ändern und diese Zellen direkt ausführen, um das Ergebnis Ihrer Änderungen sofort zu sehen. Die Zellen enthalten entweder [Markdown](https://en.wikipedia.org/wiki/Markdown)-Inhalt (wie diese Zelle) oder ausführbaren Python-Quellcode.

**Um direk loslegen zu können klicken Sie oben auf Run -> Run All Cells**  
**Einzelne ausgewählte Zellen können sie danach mit STRG + ENTER erneut ausführen uns so Abfragen individuell anpassen**

Das Notebook startet mit einem Setup der Programierumgebung. Das SPARQL tutorial startet [hier](#Tutorial).

*Zusätzliche Informationen:*  
JupyterLite stammt von JupyterLab mit dem Vorteil, vollständig browserbasiert zu sein, ohne Backend-Infrastruktur. Das bedeutet, dass die Ausführung der Zellen bei der ersten Ausführung einige Zeit in Anspruch nehmen kann. Nachfolgende Ausführungen werden aufgrund der gespeicherten Daten in Ihrem Browser-Cache viel schneller sein.

Wenn Sie mit mehr über ide Handhabung von Jupyter-Notebooks nicht wissen wollen, finden Sie hier zwei nützliche Ressourcen:

- [Die JupyterLab-Schnittstelle] (https://jupyterlab.readthedocs.io/en/stable/user/interface.html)
- [Das Jupyter-Notebook] (https://jupyterlab.readthedocs.io/en/stable/user/notebook.html)

## Setup

### Installieren und importieren der erforderlichen Module

Das Abfragen eines SPARQL-Endpunkts stellt im Grunde eine POST-Anforderung an die entsprechende Endpunkt-URL. Da JupyterLite im Moment keine Unterstützung für Pythons `requests`-Modul hat, wird die JavaScript-Fetch-API verwendet (mit einigen Tricks). Dazu müssen folgende Module importiert werden:

In [2]:
%pip install -q folium

In [3]:
import json
import pandas as pd
from pyodide.ffi import to_js
from IPython.display import JSON, HTML
from js import Object, fetch
from io import StringIO

### Hauptabfragefunktion definieren

Da die JavaScript-Abruf-API asynchron ist, muss die entsprechende Python-Funktion `query` als `async` deklariert werden. Diese Funktion ermöglicht die Abfrage der 3 staatlichen Triple Stores der Schweiz.

In [4]:
async def query(query_string, store = "L"):
    
    # three Swiss triplestores
    if store == "F":
        address = 'https://fedlex.data.admin.ch/sparqlendpoint'
    elif store == "G":
        address = 'https://geo.ld.admin.ch/query'
    else:
        address = 'https://ld.admin.ch/query'
    
    # try the Post request with help of JS fetch
    # the creation of the request header is a little bit complicated because it needs to be a 
    # JavaScript JSON that is made within a Python source code
    try:
        resp = await fetch(address,
          method="POST",
          body="query=" + query_string,
          credentials="same-origin",
          headers=Object.fromEntries(to_js({"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", 
                                            "Accept": "text/csv" })),
        )
    except:
        raise RuntimeError("fetch failed")
    
    
    if resp.ok:
        res = await resp.text()
        # ld.admin.ch throws errors starting with '{"message":'
        if '{"message":' in res:
            error = json.loads(res)
            raise RuntimeError("SPARQL query malformed: " + error["message"])
        # geo.ld.admin.ch throws errors starting with 'Parse error:'
        elif 'Parse error:' in res:
            raise RuntimeError("SPARQL query malformed: " + res)
        else:
            # if everything works out, create a pandas dataframe from the csv result
            df = pd.read_csv(StringIO(res))
            return df
    else:
        # fedlex.data.admin.ch throws error with response status 400
        if resp.status == 400:
            raise RuntimeError("Response status 400: Possible malformed SPARQL query. No syntactic advice available.")
        else:
            raise RuntimeError("Response status " + resp.status)

Wenn Sie an Einzelheiten zur Verwendung der JavaScript-Abruf-API in JupyterLite interessiert sind, stehen Ihnen hier weitere Informationen zur Verfügung:

- https://pyodide.org/en/stable/usage/faq.html#how-can-i-use-fetch-with-optional-arguments-from-python
- https://github.com/jupyterlite/jupyterlite/discussions/412
- https://lwebapp.com/en/post/pyodide-fetch

### Anzeige Funktion definieren

Zeigt den aus der SPARQL-Abfrage resultierenden Pandas-Dataframe als HTML mit anklickbaren Links an:

In [5]:
def display_result(df):
    df = HTML(df.to_html(render_links=True, escape=False))
    display(df)

# Tutorial

* Einleitung Linked Data
* Einleitung Datenmodell SR (Metadaten)
* Einleitung SPARQL (Sparql endpoint)
* 1. Abfrage: Liste aller SR mit titel short titel, datum, SR Nummer, link
* 2. Abfrage: Versionen der Bundesverfassung mit Datei
* Welche SR werden durch das neueste AS geändert

## Linked Data

Linked Data bedeutet, dass die Daten nicht als Tabellen sondern als Netwerke, sogenannte Knowlede-Graphs vabgespeichert sind.  
Einzelne Informationseinheiten sind über sogenante **Triples** miteinander verbunden. Triples folgen der Grammatikalischen Struktur **Subjekt -> Prädikat -> Objekt** und können auch als grammatikalischer Satz verstanden werden. 

Die Information "**Der Apfel ist grün**" wird also mit dem Tripel **Apfel -> ist -> grün** ausgedrückt. Alle Teile eines Tripples sind dabei durch weitere Eigenschaften definiert und beschrieben die wiederum in Form von Triples beschrieben sind. Diese vielseitigen Verknüpfungen führen zu einer Netzwerkstruktur.

Eine weitere wichtige Komponente von Linked Data ist, dass alle Teile eines Tripples als URI (Universal Resource Identifier), also als eine Https-Webadresse angegeben sind, und ihre jeweiligen Eigenschaften über diese Webadresse abrufbar sind. Das ermöglicht die Verknüpfung von Daten aus verschieden Graphen und Datenbanken zu einem **Web of Knowledge**.

Im folgenden Tutorial werden wir die Nutzung von Linked Data anhand der Fedlex-Datenbank Schritt für Schritt aufzeigen. Für weitere allgemeine Informationen über Linked Data siehe z.B: https://www.youtube.com/watch?v=ON0wf0SEPx8&list=PLoOmvuyo5UAfY6jb46jCpMoqb-dbVewxg 

## Daten und Datenmodell auf Fedlex

### Datenmodell jolux

Die Datenbank von Fedlex besteht auf dem Datenmodel (auch Ontologie genannt) **jolux**, das uhrsprünglich aus Luxemburg kommt und inzwischen von der Schweiz und Luxemburg gemeinsam weiterentwickelt wird. In Linked Data definiert ein Datenmodell welche Art von Objekten es gibt, wie diese in Beziehung stehen können und mit welchem Vokabular (URIs) die Objekte und Beziehungen benannt werden.  
Weitere Informationen über das Datenmodell jolux finden sie hier: https://fedlex.data.admin.ch/de-CH/home/models

### Amtliche Sammlung

In der **Amtlichen Sammlung (AS)** des Bundesrechts werden die neuen und geänderten Erlasse, Verträge und Beschlüsse chronologisch veröffentlicht. Typischerweise haben Beschlüsse die Form eines Mantelerlasses, in denen sowohl neue Gesetze erlassen werden, als auch schon bestehende geändert oder aufgehoben werden können. Jede Veröffentlichung in der Amtlichen Sammlung erhält eine eindeutige AS-Nummer (z.B. [AS 2021 654](https://www.fedlex.admin.ch/eli/oc/2021/654))

### Systematische Rechtssammlung

Die **Systematische Rechtssammlung (SR)** des Bundesrechts stellt die consolidierte Fassung des aktuell gültigen Rechts basierend auf der Amtlichen Sammlung dar. Jeder Gesetzestext bekommt eine eindeutige SR-Nummer (z.B. [SR 101](https://www.fedlex.admin.ch/eli/cc/1999/404) für die Bundesverfassung), welche sich nicht auf eine bestimmte Version eines Gesetzes bezieht, sondern auf das Gesetz im Allgemeinen. Erscheint ein neuer Erlass in der AS der Teile des entsprechenden Gesetzes revidiert (z.B. Teile der Bundesverfassung durch [AS 2022 241](https://www.fedlex.admin.ch/eli/oc/2022/241)), so werden diese Änderungen in der Systematischen Rechtssammlung consolidiert und unter gleichbleibender SR-Nummer veröffentlicht (im Beispiel weiterhin SR 101).   

### Metadaten und Triples

Mit welchen Linked Data Triples ein bestimmter Eintrag in der AS oder SR in der Datenbank beschrieben ist, lässt sich über den [Metadaten-Explorer](https://fedlex.data.admin.ch/de-CH/metadata) anzeigen. Dazu müssen die entsprechenden Linked Data URIs eingegeben werden.

Die Linked Data URI, eines AS oder SR Eintrags, ist dabei analog zur Web-URL:
* Web-URL: https://www.fedlex.admin.ch/eli/cc/1999/404
* Linked Data URI: https://fedlex.data.admin.ch/eli/cc/1999/404

Die URI eines AS oder SR Eintrags lässt sich auch über die Web-Darstellung finden in dem man auf das Ketten-Symbol direkt neben "Alles einblenden" klickt:
<br><img src="img/sr101.jpg" width="500px">

[In den Metadaten-Explorer eingegeben](https://fedlex.data.admin.ch/de-CH/metadata?value=https:%2F%2Ffedlex.data.admin.ch%2Feli%2Fcc%2F1999%2F404) zeigt die oben genannte URI die abgespeicherten Eigenschaften der Bundesverfassung.
<br><img src="img/metadata.png" width="500px">

### Abkürzungen

URIs sind analog zu Webadressen aufgebaut und operieren auf sogenannten Namensräumen. Alle SR und AS Einträge lassen sich beispielsweise im Namensraum `https://fedlex.data.admin.ch/eli/` finden. Um die Lesbarkeit zu verbessern, können Abkürzungen definiert werden z.B. `fedlex`. Aus `https://fedlex.data.admin.ch/eli/cc/1999/404` wird dann `fedlex:cc/1999/404`.

Für die weiteren Teile des Tutorials definieren wir folgenden Abkürzungen:
* `fedlex` : `https://fedlex.data.admin.ch/eli/`; Namensraum für AS und SR Einträge
* `jolux` : `http://data.legilux.public.lu/resource/ontology/jolux#`; Vokabular für das jolux Datenmodell
* `skos` : `http://www.w3.org/2004/02/skos/core#`; Externes Vokabular für allgemeine Eigenschaften
* `rdf` : `http://www.w3.org/1999/02/22-rdf-syntax-ns#`; Externes Vokabular für allgemeine Metadaten

Mehr zum Aufbau der URIs auf Fedlex finden sie [hier](https://fedlex.data.admin.ch/de-CH/home/convention)

## Das Datenmodell erforschen und die erste SPARQL Abfrage

Um die gespeicherten Daten zur Bundesverfassung zu erhalten, gibt es vier Möglichkeiten:

- Die HTML Version unter https://www.fedlex.admin.ch/eli/cc/1999/404/de
- Den Metadaten-Explorer unter https://fedlex.data.admin.ch/de-CH/metadata?value=https://fedlex.data.admin.ch/eli/cc/1999/404
- Die Abfrage der weiteren Daten mit SPARQL, einer Srache zum Abfragen von Linked Data Datenbanken (so genannten Triple Stores), ähnlich zu SQL.

Die Website https://fedlex.admin.ch basiert auf Daten aus dem Triple Store. Informationen, die auf den HTML-Seiten von Fedlex zu sehen sind, sind also auch maschinenlesbar via SPARQL Abfrage verfügbar. Nachfolgend soll Schritt für Schritt das Datenmodell von Fedlex eingeführt werden und die entsprechenden SPARQL Queries aufgezeigt werden, um auf diese Daten maschinenlesbar zugreifen zu können.

### SPARQL

SPARQL ist eine Query-Sprache für Linked Data Triple Stores. Für eine allgemeine Einführung in SPARQL siehe z.B.: https://jena.apache.org/tutorials/sparql.html

Abfragen (engl. Queries) können entweder direkt über ein SPARQL Interface (https://fedlex.data.admin.ch/de-CH/sparql) abgefragt werden, oder als HTTP-POST Request an ein SPARQL-Endpoint (https://fedlex.data.admin.ch/sparqlendpoint) gesendet werden.

Die letztere Methode erlaubt es eigene Anwendungen zu bauen, die automatisch aktuelle Daten von Fedlex abfragen können. Für dieses Tutorial verwenden wir diese Methode. Die Abfragen sind jedoch in beiden Fällen identisch. Für die Abfrage über das Interface kopieren sie einfach den Teil zwischen den `"""`.

### Eigenschaften der Bundesvefassung

Eine SPARQL Abfrage funktioniert grob, in dem man Aussagen formuliert, und die Teile die man wissen möchte, durch Variablen ersetzt. Variablen sind in SPARQL durch ein vorangestelltes `?` gekennzeichnet.

Wenn wir die Eigenschaften der Bundesverfassung wissen wollen, also alle Triples (oder Aussagen) in denen die Bundesverfassung als Subjekt erscheint, dann lautet die entsprechende SPARQL Query wie folgt:

In [20]:
df = await query("""

# SELECT beschreibt welche Variablen zurückgegeben werden sollen.
# Mit DISTINCT werden allfällige doppelte Ergebnisse aussortiert.

SELECT DISTINCT ?prädikat ?objekt WHERE {
    
    # Die Aussage beginnt mit der URI der Bundesverfassung als Subjekt, setzt das Prädikat und Objekt als Variablen und endet mit einem Punkt.
    <https://fedlex.data.admin.ch/eli/cc/1999/404> ?prädikat ?objekt .
} 

""", "F")

display_result(df)

,prädikat,objekt
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#Work
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#WorkAtOj
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#ConsolidationAbstract
3,http://data.legilux.public.lu/resource/ontology/jolux#basicAct,https://fedlex.data.admin.ch/eli/oc/1999/404
4,http://data.legilux.public.lu/resource/ontology/jolux#parliamentDraftId,1996.091
5,http://data.legilux.public.lu/resource/ontology/jolux#typeDocument,https://fedlex.data.admin.ch/vocabulary/resource-type/55
6,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/de
7,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/en
8,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/fr
9,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/it


Das Ergebniss sind die URIs oder Werte (Literals) der Prädikate und Objekte die in allen abgespeicherten Triples mit der Bundesverfassung als Subjekt vorkommen.

Wir sehen, dass der Eintrag vom `rdf:type` `jolux:ConsolidationAbstract` ist. Dieser Type beschreibt Einträge die einen SR-Eintrag darstellen, also ein Gesetz auf Abstrakter Ebene repräsentieren, anstatt einer bestimmten Version des Textes.

### Expressions: Sprachversionen

Da Alle Texte in mindestens vier, manchmal fünf Sprachen existieren, gibt es von jedem Eintrag verschiedene Sprachversionen. Diese sind vom `rdf:type` `jolux:Expression` und sind durch die Eigenschaft `jolux:isRealizedBy` mit dem Sprachübergreifenden Eintrag verknüpft.

Mit einem Klick auf die URI der deutschen Sprachversion (https://fedlex.data.admin.ch/eli/cc/1999/404/de) sehen wir, dass wir hier sowohl Titel, Abkürzung, als auch die SR Nummer auf deutsch finden. 

<br><br><img src=img/expression.svg>
<br><br>

Ausgehend von der URI der Bundesverfassung können wir diese Informationen also abfragen mit:

In [22]:
df = await query("""

# Definition von Abkürzungen zur besseren Lesbarkeit
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?SR_Nummer ?Titel ?Abkürzung WHERE {
    
    # Die erste Aussage wählt alle Expressions (Sprachversionen) aus die durch jolux:isRealizedBy mit der Bundesverfassung verknüpft sind.
    <https://fedlex.data.admin.ch/eli/cc/1999/404> jolux:isRealizedBy ?expression .
    
    # Die zweite Aussage grenzt die Expressions ein zu der, die als Sprache Deutsch hat.
    ?expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    # Die nächsten Aussagen fragen die gewünschen Daten ab, jeweils mit der Expression als Subjekt (möglich durch das Beenden der Aussage mit ; anstatt .)
    ?expression jolux:title ?Titel ;
                jolux:titleShort ?Abkürzung ; 
                jolux:historicalLegalId ?SR_Nummer .
} 

""", "F")

display_result(df)

,SR_Nummer,Titel,Abkürzung
0,101,Bundesverfassung der Schweizerischen Eidgenossenschaft vom 18. April 1999,BV


### Liste aller SR Einträge

Basierend auf dem was wir bereits gelernt haben können wir nun ganz einfach eine Liste aller SR-Einträge und die dazugehörigen Metadaten abfragen indem wir einfach die URI der Bundesverfassung durch eine Variable ersetzen die vom `rdf:type` `jolux:ConsolidationAbstract` sein soll (also einen SR Eintrag darstellt):

In [26]:
df = await query("""

# Definition von Abkürzungen zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?SR_Nummer ?Titel ?Abkürzung ?SR_URI WHERE {
    
    # Alle SR Einträge und ihre Expressions auswählen
    ?SR_URI rdf:type jolux:ConsolidationAbstract .
    ?SR_URI jolux:isRealizedBy ?expression .
    
    ?expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    ?expression jolux:title ?Titel ;
                jolux:titleShort ?Abkürzung ; 
                jolux:historicalLegalId ?SR_Nummer .
} 

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

,SR_Nummer,Titel,Abkürzung,SR_URI
0,734.000,"Bundesgesetz vom 24. Juni 1902 betreffend die elektrischen Schwach- und Starkstromanlagen (Elektrizitätsgesetz, EleG)",EleG,https://fedlex.data.admin.ch/eli/cc/19/259_252_257
1,836.100,Bundesgesetz vom 20. Juni 1952 über die Familienzulagen in der Landwirtschaft (FLG),FLG,https://fedlex.data.admin.ch/eli/cc/1952/823_843_839
2,916.350,"Beschluss der Bundesversammlung vom 29. September 1953 über Milch, Milchprodukte und Speisefette (Milchbeschluss)",MB,https://fedlex.data.admin.ch/eli/cc/1953/1109_1132_1172
3,613.100,Bundesgesetz vom 19. Juni 1959 über den Finanzausgleich unter den Kantonen,FAG,https://fedlex.data.admin.ch/eli/cc/1959/931_961_953
4,831.201,Verordnung vom 17. Januar 1961 über die Invalidenversicherung (IVV),IVV,https://fedlex.data.admin.ch/eli/cc/1961/29_29_29
5,746.100,"Bundesgesetz vom 4. Oktober 1963 über Rohrleitungsanlagen zur Beförderung flüssiger oder gasförmiger Brenn- oder Treibstoffe (Rohrleitungsgesetz, RLG)",RLG,https://fedlex.data.admin.ch/eli/cc/1964/99_95_95
6,416.000,Bundesgesetz vom 19. März 1965 über die Gewährung von Beiträgen an die Aufwendungen der Kantone für Ausbildungsbeihilfen (Ausbildungsbeihilfengesetz),StipG,https://fedlex.data.admin.ch/eli/cc/1965/477_481_475
7,741.410,Verordnung vom 27. August 1969 über Bau und Ausrüstung der Strassenfahrzeuge (BAV),BAV,https://fedlex.data.admin.ch/eli/cc/1969/821_841_839
8,952.020,"Verordnung vom 17. Mai 1972 über die Banken und Sparkassen (Bankenverordnung, BankV)",BankV,https://fedlex.data.admin.ch/eli/cc/1972/821_832_752
9,514.511,Verordnung vom 10. Januar 1973 über das Kriegsmaterial,VKM,https://fedlex.data.admin.ch/eli/cc/1973/116_114_120


Der Übersicht halber wurde die Ausgabe mit dem Keyword `LIMIT` auf die ersten 10 Einträge begrenzt.

# Ab hier Work in Progress

### Die Bundesverfassung als PDF

Als nächstes Ziel soll ein Link auf die PDF Datei der Bundesverfassung abgefragt werden. Dazu muss etwas tiefer ins Datenmodell eingestigen werden. Ein Blick auf die ausgehenden Verbindungen der Bundesverfassung lässt die Vermutung zu, dass sich evt. unter der URI https://fedlex.data.admin.ch/eli/cc/1999/404/de etwas verbergen würden. Ein Klick auf die URI (die einem automatisch zum Metadatenviewer bringt), zeigt aber auf, dass dies eher eine Sackgasse ist. Interessanter sind die Links [jolux:isMemberOf](http://data.legilux.public.lu/resource/ontology/jolux#isMemberOf), da wäre bspw. https://fedlex.data.admin.ch/eli/cc/1999/404/20220213, die Ansicht im [Metadatenviewer](https://fedlex.data.admin.ch/de-CH/metadata?value=https:%2F%2Ffedlex.data.admin.ch%2Feli%2Fcc%2F1999%2F404%2F20220213) zeigt, dass es dort über [jolux:isRealizedBy](http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy) die URI https://fedlex.data.admin.ch/eli/cc/1999/404/20220213/de zu finden ist. Weiters Vorarbeiten im Datenmodell bringt die in der folgenden Grafik illustrierten Zusammenhänge zum Vorschein: <br><br><img src=img/sr1.svg>
<br><br>
Dementsprechend kann nun die folgende SPARQL Query für die PDF aller verschiedener Versionen der Bundesverfassung erstellt werden:

In [18]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?link WHERE {

    <https://fedlex.data.admin.ch/eli/cc/1999/404> ^jolux:isMemberOf/jolux:isRealizedBy ?expression.
    
    ?expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU>;
        jolux:isEmbodiedBy ?manifestation.
    
    ?manifestation jolux:format <http://publications.europa.eu/resource/authority/file-type/PDF>;
        jolux:isExemplifiedBy ?link

}

""", "F")

display_result(df)

,link
0,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20090517/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20090517-de-pdf-a.pdf
1,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20051127/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20051127-de-pdf-a.pdf
2,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20080101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20080101-de-pdf-a.pdf
3,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20100307/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20100307-de-pdf-a.pdf
4,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20110101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20110101-de-pdf-a.pdf
5,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20170924/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20170924-de-pdf-a.pdf
6,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20060521/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20060521-de-pdf-a.pdf
7,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20091129/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20091129-de-pdf-a.pdf
8,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20120311/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20120311-de-pdf-a.pdf
9,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20120923/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20120923-de-pdf-a.pdf
